<a href="https://colab.research.google.com/github/AtiendriyaV/COA/blob/main/Cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2023-01-11 03:20:14--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.20.126
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.20.126|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/downloads/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2023-01-11 03:20:15--  https://developer.nvidia.com/downloads/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?igd3RsEnDz3hucuYWmGjCJ6D3cEZMpmkh5XbGLfOfj7k7U1faJf3xSDyyr4S23DuSc3cb0903GtNIdg_TKGFOvTpm

In [3]:
!apt-get update

Get:1 file:/var/cuda-repo-9-2-local  InRelease
Ign:1 file:/var/cuda-repo-9-2-local  InRelease
Get:2 file:/var/cuda-repo-9-2-local  Release [574 B]
Get:2 file:/var/cuda-repo-9-2-local  Release [574 B]
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/

In [ ]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [4]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-wuhcft6h
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-wuhcft6h
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=806b321aee2ebf9b0a35ae264a4b5d2a4a4f3da3df3eb81627d7c9f83b414e10
  Stored in directory: /tmp/pip-ephem-wheel-cache-wpbh9lgv/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin


In [5]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [6]:
%%cu
#include <stdio.h>
__global__ void Hellokernel()
{

}
main()
{
Hellokernel<<<1, 1>>>();
 printf("Hello cuda program srivani 22MAI1007\n");
return 0;
}

Hello cuda program srivani 22MAI1007



In [7]:
%%cu
#include <stdio.h>
__global__ void add(int a, int b, int *c)
{
*c = a + b;
}
int main(void)
{
int c;
int *dev_c;
cudaMalloc((void**)&dev_c, sizeof(int));
add << <1, 1 >> > (2, 7, dev_c);
cudaMemcpy(&c, dev_c, sizeof(int),
cudaMemcpyDeviceToHost);
printf("2 + 7 = %d\n", c);
cudaFree(dev_c);
return 0;
}

2 + 7 = 9



In [ ]:
%%cu
#include <stdio.h>
__global__ void vector_add(int *out_d, int *a, int *b, int n) 
	{ 
	int bx = blockIdx.x;
int by = blockIdx.y;
int tx = threadIdx.x;
int ty = threadIdx.y;
int row = by*blockDim.y + ty;
int col = bx*blockDim.x + tx;
int dim = gridDim.x*blockDim.x;
int i = row*dim + col;
out_d[i] = a[i] + b[i]; 

	 }

int main()
	{ 
	int *a, *b, *out_d,*out;
	 int *d_a, *d_b;
	 int N=6;
	int i;
	 a = (int*)malloc(sizeof(int) * N);
	 b = (int*)malloc(sizeof(int) * N);
  	 out = (int*)malloc(sizeof(int) * N);
	for (i=0;i<N;i++)
		{
		a[i]=i;
		b[i]=i*2;
		}
 	cudaMalloc((void**)&d_a, sizeof(int) * N); 
 	cudaMalloc((void**)&d_b, sizeof(int) * N); 
  cudaMalloc((void**)&out_d, sizeof(int) * N); 
	cudaMemcpy(d_a, a, sizeof(int) * N, cudaMemcpyHostToDevice); 
	cudaMemcpy(d_b, b, sizeof(int) * N, cudaMemcpyHostToDevice);
	vector_add<<<2,4>>>(out_d, d_a, d_b, N); 
  	cudaMemcpy(out, out_d, sizeof(int) * N, cudaMemcpyDeviceToHost);
	printf("Success");
  	for (i=0;i<N;i++)
		{
		printf("%d\n",out[i]);
		}
	cudaFree(d_a); 
	cudaFree(d_b); 
  	cudaFree(out_d); 
	free(a); 
	free(b); 
  	free(out); 
  return 0;
	}


Success0
3
6
9
12
15



In [ ]:
%%cu
#include <stdio.h>
__global__ void matrixMul( int* Pd, int* Md, int* Nd, int width)
{
int bx = blockIdx.x;
int by = blockIdx.y;
int tx = threadIdx.x; 
int ty = threadIdx.y;
int col = by*blockDim.y + ty;
int row = bx*blockDim.x + tx;
int Pvalue=0;
for (int k=0;k<width;++k)
	Pvalue+=Md[row*width+k]*Nd[k*width+col];
Pd[row*width+col]=Pvalue;
}

int main()
	{ 
	int *M, *N1, *Md, *Nd, *Pd, *P;
const int xb = 3; /* gridDim.x */
const int yb = 3; /* gridDim.y */
const int zb = 1; /* gridDim.z */
const int xt = 3; /* blockDim.x */
const int yt = 3; /* blockDim.y */
const int zt = 1; /* blockDim.z */
	 int N,width;
  int i;
    width=9;
   N=width*width;
	 M = (int*)malloc(sizeof(int) * N);
	 N1 = (int*)malloc(sizeof(int) * N);
  	 P = (int*)malloc(sizeof(int) * N);
	for (i=0;i<N;i++)
		{
		M[i]=i;
		N1[i]=i*2;
		}
  dim3 dimGrid(xb,yb,zb);
dim3 dimBlock(xt,yt,zt);
 	cudaMalloc((void**)&Md, sizeof(int) * N); 
 	cudaMalloc((void**)&Nd, sizeof(int) * N); 
  cudaMalloc((void**)&Pd, sizeof(int) * N); 
	cudaMemcpy(Md, M, sizeof(int) * N, cudaMemcpyHostToDevice); 
	cudaMemcpy(Nd, N1, sizeof(int) * N, cudaMemcpyHostToDevice);
	matrixMul<<<dimGrid,dimBlock>>>(Pd, Md, Nd, width); 
  	cudaMemcpy(P, Pd, sizeof(int) * N, cudaMemcpyDeviceToHost);
	printf("Success");
  	for (i=0;i<N;i++)
		{
		printf("%d\n",P[i]);
		}
	cudaFree(Md); 
	cudaFree(Nd); 
  	cudaFree(Pd); 
	free(M); 
	free(N1); 
  	free(P); 
  return 0;
	}


Success3672
3744
3816
3888
3960
4032
4104
4176
4248
9504
9738
9972
10206
10440
10674
10908
11142
11376
15336
15732
16128
16524
16920
17316
17712
18108
18504
21168
21726
22284
22842
23400
23958
24516
25074
25632
27000
27720
28440
29160
29880
30600
31320
32040
32760
32832
33714
34596
35478
36360
37242
38124
39006
39888
38664
39708
40752
41796
42840
43884
44928
45972
47016
44496
45702
46908
48114
49320
50526
51732
52938
54144
50328
51696
53064
54432
55800
57168
58536
59904
61272



In [ ]:
%%cu
#include <stdio.h>
__global__ void matrixFill ( int *x )
{
int bx = blockIdx.x;
int by = blockIdx.y;
int tx = threadIdx.x;
int ty = threadIdx.y;
int col = by*blockDim.y + ty; 
int row = bx*blockDim.x + tx; 
int dim =blockDim.x*gridDim.x;
int i = row*dim + col; 
x[i] = i;
}
int main ( int argc, char* argv[] )
{
const int xb = 2; /* gridDim.x */
const int yb = 2; /* gridDim.y */
const int zb = 1; /* gridDim.z */
const int xt = 2; /* blockDim.x */
const int yt = 2; /* blockDim.y */
const int zt = 1; /* blockDim.z */
const int n = xb*yb*zb*xt*yt*zt;
printf("allocating array of length %d...\n",n);
int *xhost = (int*)calloc(n,sizeof(int));
for(int i=0; i<n; i++) xhost[i] = -1.0;
int *xdevice;
size_t sx = n*sizeof(int);
cudaMalloc((void**)&xdevice,sx);
cudaMemcpy(xdevice,xhost,sx,cudaMemcpyHostToDevice);
dim3 dimGrid(xb,yb,zb);
dim3 dimBlock(xt,yt,zt);
matrixFill<<<dimGrid,dimBlock>>>(xdevice);
cudaMemcpy(xhost,xdevice,sx,cudaMemcpyDeviceToHost);
cudaFree(xdevice);
int *p = xhost;
for(int i1=0; i1 < xb; i1++)
for(int i2=0; i2 < yb; i2++)
for(int i3=0; i3 < zb; i3++)
for(int i4=0; i4 < xt; i4++)
for(int i5=0; i5 < yt; i5++)
for(int i6=0; i6 < zt; i6++)
printf("x[%d][%d][%d][%d][%d][%d] = %d\n",i1,i2,i3,i4,i5,i6,*(p++));
return 0;
}